<a href="https://colab.research.google.com/github/movie112/INU-DILAB/blob/main/lab_11_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab_11_2: RNN_hihello/ charseq
## contents
- 'Hihello' problem
  - 예제 진행하기 앞서 정확히 어떤 문제를 풀고자 하는지 설명
- Data setting
  - One hot encoding
  - input output 어떻게 표현되는지  
- Cross entropy loss
- Code run through
---
## 'hihello' problem
> - 'hihello' 문자열을 예측하는 모델
>   - character가 들어오면 다음 char 예측하는 모델
>   - 'h'가 들어오면 'i'를 예측, 'i'가 들어오면 'h' 예측

## data setting
<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FP5fM0%2FbtqC219isUt%2F7OQbPKDKvAaBoB24GnCJk0%2Fimg.png" width="500px" height="300px"></img>
- __one-hot encoding__
<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FkRKAq%2FbtqC3qAYiFK%2FyyK63wqTXjCsNV7TXnd2M1%2Fimg.png" width="500px" height="300px"></img>
  - 다음 문자를 예측하는 모델이기 때문에 one_hot encoding에서 마지막'o'는 제외
  - y_data는 처음 문자 'h' 제외

> ## cross entropy loss
> <img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FxRLTH%2FbtqC0JuNiuR%2FkWLLk5KCzK5DcUaHILLcvK%2Fimg.png" width="500px" height="300px"></img>
> - categorical output 예측에 주로 사용
>   - 일반적으로 output을 softmax를 이용하여 확률값으로 해석
>   - 확률값을 정답 카테고리에 대해 최대한 올림: CEL(cross entropy loss) 역할
> - loss = criterion(모델의 output, 정답label)
## code run through






In [1]:
import torch
import torch.optim as optim
import numpy as np

In [2]:
# Random seed to make results deterministic and reproducible
torch.manual_seed(0)

- (1) code run through(hihello)

In [4]:
# make dictionary 데이터 준비 과정(모델 생성 전)
char_set = ['h', 'i', 'e', 'l', 'o']  # char list 정의
# hyper parameters 정의
dic_size = len(char_dic)
hidden_size = len(char_dic)   # 다른 숫자여도 상관 없음
learning_rate = 0.1

{'f': 0, 'o': 1, 'y': 2, ' ': 3, 'a': 4, 'u': 5, 'n': 6, 'i': 7, 'w': 8, 't': 9}


In [ ]:
# data setting ; one hot encoding을 통해 input data 제작
x_data = [[0, 1, 0, 2, 3, 3]]   # char들을 index로 표현한 것
x_one_hot = [[[1, 0, 0, 0, 0], 
              [0, 1, 0, 0, 0], 
              [0, 0, 1, 0, 0], 
              [0, 0, 0, 1, 0], 
              [0, 0, 0, 0, 1]]]
y_data = [[1, 0, 2, 3, 3, 4]]   # i h e l l o

In [ ]:
# transform as torch tensor variable
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

- (2) code run through(charseq): 데이터 준비과정 일반화

In [6]:
# 어떤 문자열이든 동작
sample = " if you want you"
# make dictionary
char_set = list(set(sample))    # set 함수: 중복 제거한 문자 list 생성

# 특정 char를 주면 그 index로 알아서 찾아주는 dictionary
char_dic = {c: i for i, c in enumerate(char_set)} # enumerate: index, char 함께 가져옴
print(char_dic)

{'f': 0, 'o': 1, 'y': 2, ' ': 3, 'a': 4, 'u': 5, 'n': 6, 'i': 7, 'w': 8, 't': 9}


In [ ]:
# hyper parameters
dic_size = len(char_dic)
hidden_size = len(char_dic)
learning_rate = 0.1

In [ ]:
# data setting
sample_idx = [char_dic[c] for c in sample]    # char_dic으로 index로 변환
x_data = [sample_idx[:-1]]
x_one_hot = [np.eye(dic_size)[x] for x in x_data]    # np.eye: 단위행렬 만들어줌(one hot encoding)
y_data = [sample_idx[1:]]

In [ ]:
# transform as torch tensor variable
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

- RNN 생성, 학습

In [ ]:
# declare RNN
rnn = torch.nn.RNN(dic_size, hidden_size, batch_first=True) #true: batch_dim이 가장 앞으로

In [ ]:
# loss & optimizer setting
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(rnn.parameters(), learning_rate)

In [ ]:
# start training
for i in range(50):
    optimizer.zero_grad() # 꼭 해주어야 함: 기존 gradient에 축적 방지
    outputs, _status = rnn(X)
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))  # shape batch_dim이 앞에 오도록 재배치 -> loss 계산
    loss.backward()   # backpropagation -> gradient 계산
    optimizer.step()  # update

    # 실제 모델이 어떻게 예측했는지 확인하는 코드
    result = outputs.data.numpy().argmax(axis=2) # argmax:어떤 char가 가장 가능성있는지에 대한 숫자 중 가장 큰 숫자가 있는 index를 가져옴
    result_str = ''.join([char_set[c] for c in np.squeeze(result)])
    print(i, "loss: ", loss.item(), "prediction: ", result, "true Y: ", y_data, "prediction str: ", result_str)